In [9]:
l = [0,1,2,3]

In [10]:
l[0:1]

[0]

In [11]:
slice(2, 4, 0)

slice(2, 4, 0)

In [12]:
import numpy
import scipy.sparse
pc = 0.5
pr = 0.4
expect = [
    [(1/3**pc) / (1/3**pc + 1/2**pc + 1)**pr,
     (1/2**pc) / (1/3**pc + 1/2**pc + 1)**pr,
     1 / (1/3**pc + 1/2**pc + 1)**pr],
    [(1/3**pc) / (1/3**pc + 1/2**pc)**pr,
     (1/2**pc) / (1/3**pc + 1/2**pc)**pr,
     0],
    [(1/3**pc) / (1/3**pc)**pr, 0, 0],
]
expect = numpy.array(expect, dtype='float64')
expect

array([[ 0.41488351,  0.50812645,  0.71859932],
       [ 0.5223379 ,  0.63973066,  0.        ],
       [ 0.71922309,  0.        ,  0.        ]])

In [25]:
expect.argmax(axis=0)

array([2, 1, 0])

In [20]:
numpy.diag(expect.diagonal())

array([[ 0.41488351,  0.        ,  0.        ],
       [ 0.        ,  0.63973066,  0.        ],
       [ 0.        ,  0.        ,  0.        ]])

In [15]:
scipy.sparse.diags(expect.diagonal())

<3x3 sparse matrix of type '<class 'numpy.float64'>'
	with 3 stored elements (1 diagonals) in DIAgonal format>

In [17]:
numpy.zeros_like(expect) - scipy.sparse.diags(expect.diagonal())

matrix([[-0.41488351,  0.        ,  0.        ],
        [ 0.        , -0.63973066,  0.        ],
        [ 0.        ,  0.        ,  0.        ]])

In [27]:
import hetio.readwrite
import collections

url = 'https://github.com/dhimmel/hetio/raw/{}/{}'.format(
    'c1d47a942f2387cde4ee40c67b7dc5bd93b598f8',
    'test/data/bupropion-CbGpPWpGaD-subgraph.json.xz',
)
graph = hetio.readwrite.read_graph(url)
metapath = graph.metagraph.metapath_from_abbrev('CbGpPWpGaD')

In [49]:
def pairwise(iterable):
    """
    Recipe from itertools docs at
    https://docs.python.org/3/library/itertools.html

    s -> (s0,s1), (s1,s2), (s2, s3), ...
    """
    a, b = itertools.tee(iterable)
    next(b, None)
    return zip(a, b)

In [48]:
metanodes = metapath.get_nodes()
metanode_to_indexes = collections.OrderedDict()
for i, metanode in enumerate(metanodes):
    indexes = metanode_to_indexes.setdefault(metanode, [])
    indexes.append(i)

# If there are no duplicate metanodes, use DWWC
print(max(map(len, metanode_to_indexes.values())))

# Ensure no overlapping metanode duplications
last_stop = -1
for metanode, indexes in metanode_to_indexes.items():
    if len(indexes) == 1:
        continue
    if max(indexes) <= last_stop:
        msg = ('Metapath f{metapath} contains overlapping'
               'segments with duplicate metanodes.')
        raise NotImplementedError(msg)
    last_stop = max(indexes)



2


IndexError: tuple index out of range

In [54]:
split_at = [0]
for metanode, indexes in metanode_to_indexes.items():
    if len(indexes) == 1:
        continue
    split_at.append(min(indexes))
    split_at.append(max(indexes))
split_at.append(len(metapath))
split_at

[0, 1, 3, 4]

In [57]:
import itertools
list(pairwise(split_at))

[(0, 1), (1, 3), (3, 4)]

In [68]:
segments = (metapath[start:stop] for start, stop in pairwise(split_at))
segments = [graph.metagraph.get_metapath(metaedges) for metaedges in segments]
segments

[CbG, GpPWpG, GaD]

In [59]:
metapath

CbGpPWpGaD

In [61]:
[(start, stop) for start, stop in pairwise(split_at)]

[(0, 1), (1, 3), (3, 4)]

In [90]:
def pairwise(iterable):
    """
    Yield consequitive pairs of items from the iterable, but skip pairs where
    the items are equal.
    Modified from recipe in itertools docs at
    https://docs.python.org/3/library/itertools.html

    s -> (s0,s1), (s1,s2), (s2, s3), ...
    """
    a, b = itertools.tee(iterable)
    next(b, None)
    for a, b in zip(a, b):
        if b != a:
            yield a, b


In [92]:
metapath = metagraph.metapath_from_abbrev('CbGpPW')
metagraph = graph.metagraph
metanodes = metapath.get_nodes()
metanode_to_indexes = collections.OrderedDict()
for i, metanode in enumerate(metanodes):
    indexes = metanode_to_indexes.setdefault(metanode, [])
    indexes.append(i)

# Ensure no overlapping metanode duplications
last_stop = -1
for metanode, indexes in metanode_to_indexes.items():
    if len(indexes) == 1:
        continue
    if min(indexes) <= last_stop:
        msg = ('Metapath f{metapath} contains overlapping'
               'segments with duplicate metanodes.')
        raise NotImplementedError(msg)
    last_stop = max(indexes)

# Split into segments
split_at = [0]
range_to_duplicate = collections.OrderedDict()
for metanode, indexes in metanode_to_indexes.items():
    if len(indexes) == 1:
        continue
    start = min(indexes)
    stop = max(indexes)
    range_to_duplicate[(start, stop)] = metanode
    split_at.append(start)
    split_at.append(stop)
split_at.append(len(metapath))
ranges = list(pairwise(split_at))
segments = (metapath[start:stop] for start, stop in ranges)
segments = [metagraph.get_metapath(metaedges) for metaedges in segments]
duplicates = [range_to_duplicate.get(range, None) for range in ranges]
segments, duplicates

([CbGpPW], [None])

In [95]:
def pairwise(iterable):
    """
    Yield consequitive pairs of items from the iterable, but skip pairs where
    the items are equal.
    Modified from recipe in itertools docs at
    https://docs.python.org/3/library/itertools.html

    s -> (s0,s1), (s1,s2), (s2, s3), ...
    """
    a, b = itertools.tee(iterable)
    next(b, None)
    for a, b in zip(a, b):
        if b != a:
            yield a, b


def get_segments(graph, metapath):
    """
    Break metapath into sub-metapaths that have at most one duplicate
    node type.
    """
    metagraph = graph.metagraph
    metanodes = metapath.get_nodes()
    metanode_to_indexes = collections.OrderedDict()
    for i, metanode in enumerate(metanodes):
        indexes = metanode_to_indexes.setdefault(metanode, [])
        indexes.append(i)

    # If there are no duplicate metanodes, use DWWC
    if max(map(len, metanode_to_indexes.values())) < 2:
        return dwwc(graph, metapath, damping)

    # Ensure no overlapping metanode duplications
    last_stop = -1
    for metanode, indexes in metanode_to_indexes.items():
        if len(indexes) == 1:
            continue
        if min(indexes) <= last_stop:
            msg = ('Metapath f{metapath} contains overlapping'
                   'segments with duplicate metanodes.')
            raise ValueError(msg)
        last_stop = max(indexes)

    # Find indices to split at
    split_at = [0]
    range_to_duplicate = collections.OrderedDict()
    for metanode, indexes in metanode_to_indexes.items():
        if len(indexes) == 1:
            continue
        start = min(indexes)
        stop = max(indexes)
        range_to_duplicate[(start, stop)] = metanode
        split_at.append(start)
        split_at.append(stop)
    split_at.append(len(metapath))

    # Split at indices
    ranges = list(pairwise(split_at))
    segments = (metapath[start:stop] for start, stop in ranges)
    segments = [metagraph.get_metapath(metaedges) for metaedges in segments]
    duplicates = [range_to_duplicate.get(range, None) for range in ranges]
    return segments, duplicates


In [97]:
metapath = metagraph.metapath_from_abbrev('CbGpPWpG')
get_segments(graph, metapath)

([CbG, GpPWpG], [None, <hetio.hetnet.MetaNode at 0x7f78f5ec42b0>])

In [5]:
import matrix

In [3]:
from degree_weight import dwpc

ImportError: attempted relative import with no known parent package

In [98]:
def get_bupropion_subgraph():
    """
    Read the bupropion and nicotine dependence Hetionet v1.0 subgraph.
    """
    url = 'https://github.com/dhimmel/hetio/raw/{}/{}'.format(
        'c1d47a942f2387cde4ee40c67b7dc5bd93b598f8',
        'test/data/bupropion-CbGpPWpGaD-subgraph.json.xz',
    )
    return hetio.readwrite.read_graph(url)

graph = get_bupropion_subgraph()
compound = 'DB01156'  # Bupropion
disease = 'DOID:0050742'  # nicotine dependences
metapath = graph.metagraph.metapath_from_abbrev('CbGpPWpGaD')
out = dwpc(graph, metapath, damping=0)
print(out)
rows, cols, pc_matrix = dwpc(graph, metapath, damping=0)


NameError: name 'dwpc' is not defined